In [ ]:
def landsat_scene_list(aoi, start_date, end_date):
    '''Creates a list of Landsat 8, level 1, tier 1
    scenes that overlap with an aoi and are captured
    within a specified date range.

    Parameters
    ----------
    aoi : str
        The path to a shape file of an aoi with geometry.

    start-date : str
        The first date from which to start looking for
        Landsat image capture in the format yyyy-mm,dd, 
        e.g. '2017-10-01'.
        
    end-date : str
        The last date from which to looking for
        Landsat image capture in the format yyyy-mm,dd, 
        e.g. '2017-10-31'.

    Returns
    -------
    wrs : shapefile
        A catalog of Landsat 8 scenes.
    
    scenes : geopandas.geodataframe.GeoDataFrame
        A dataframe containing the information
        of Landsat 8, Level 1, Tier 1 scenes that 
        overlap with the aoi.
    '''
    # Download Landsat 8 catalog from USGS (get_data auto unzips)
    USGS_url = 'https://landsat.usgs.gov/sites/default/files/documents/WRS2_descending.zip'
    et.data.get_data(url=USGS_url, replace=True)

    # Open Landsat catalog
    wrs = gpd.GeoDataFrame.from_file(os.path.join('data', 'earthpy-downloads',
                                              'WRS2_descending',
                                              'WRS2_descending.shp'))
    
    # Find polygons that intersect Landsat catalog and aoi 
    wrs_intersection = wrs[wrs.intersects(aoi.geometry[0])]
    
    # Calculated paths and rows 
    paths, rows = wrs_intersection['PATH'].values, wrs_intersection['ROW'].values
    
    # Iterate through each Polygon of paths and rows intersecting the area
    for i, row in wrs_intersection.iterrows():
        # Create a string for the name containing the path and row of this Polygon
        name = 'path: %03d, row: %03d' % (row.PATH, row.ROW)
        
    # Removing scenes with small amounts of overlap using threshold of intersection area
    b = (paths > 23) & (paths < 26)
    paths = paths[b]
    rows = rows[b]
    
#     # Path(s) and row(s) covering the intersection
#     ############################ WHY NOT PRINTING? ###################################
#     for i, (path, row) in enumerate(zip(paths, rows)):
#         print('Image', i+1, ' - path:', path, 'row:', row)
    
    # Check scene availability in Amazon S3 bucket list of Landsat scenes
    s3_scenes = pd.read_csv('http://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz', 
                        compression='gzip', parse_dates=['acquisitionDate'], 
                        index_col=['acquisitionDate'])
    
    # Capture only Landsat T1 scenes within dates of interest
    scene_mask = (s3_scenes.index > start_date) & (s3_scenes.index  <= end_date) 
    scene_dates = s3_scenes.loc[scene_mask]
        
    scene_product = scene_dates[scene_dates['productId'].str.contains("_T1")]
    
    # Geodataframe of scenes with <5% cloud cover, the url to retrieve them
    #############################row.ROW and row.PATH will need to be fixed##################
    scenes = scene_product[(scene_product.path == row.PATH) & 
                       (scene_product.row == row.ROW) & 
                       (scene_product.cloudCover <= 5)]

    return wrs, scenes